In [1]:
import igl
import numpy as np
import meshplot as mp

import pythreejs as p3s
import ipywidgets as iw
from IPython.display import clear_output

EPS  = 0.00000000001
EPS1 = 0.0000001
EPS2 = 0.00001

# Load meshes and display

In [2]:
with np.load('data/octa_sphere_5.npz') as npl:
    v_s, f_s = npl['v'], npl['f']

#Mesh
#v_m, f_m = igl.read_triangle_mesh("data/deer.obj")
v_m, f_m = igl.read_triangle_mesh("data/cat.obj")

#Cage
#v_c, f_c = igl.read_triangle_mesh("data/deer_cage.obj")
v_c, f_c = igl.read_triangle_mesh("data/cat_cage.obj")

plt = mp.plot(v_m, f_m)

lines_start = np.vstack( [v_c[f_c[:,0]], v_c[f_c[:,1]], v_c[f_c[:,2]]] )
lines_end =  np.vstack( [v_c[f_c[:,1]], v_c[f_c[:,2]], v_c[f_c[:,0]]] )

plt.add_lines( lines_start, lines_end, shading={"line_color": "red"});
plt.add_points(v_c, shading={"point_color": "green", "point_size": 0.5})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.317513…

2

# Green coordinates

In [3]:
#deformations functions
n = igl.per_face_normals( v_c, f_c, np.array([1.0, 0, 0]) ) 

def gcTriInt(p, v1, v2, eta):
    cos_alpha =  ((v2-v1)@(p-v1)) / (np.linalg.norm(v2-v1)*np.linalg.norm(p-v1)) 
    cos_beta  =  ((v1-p)@(v2-p)) / (np.linalg.norm(v1-p)*np.linalg.norm(v2-p)) 
    if abs(cos_alpha) > (1 - EPS) or abs(cos_beta) > (1 - EPS): return 0
    #Calculate alpha
    alpha = np.arccos( cos_alpha )
    if abs(alpha - np.pi) < EPS or abs(alpha) < EPS: return 0
    #Calculate beta
    beta  = np.arccos( cos_beta )
    if abs(beta - np.pi) < EPS or abs(beta) < EPS: return 0
    
    l =  (np.linalg.norm(p-v1)**2)*(np.sin(alpha)**2)
    c = np.linalg.norm(p - eta)**2
    
    I = np.zeros(2)
    theta_list = [np.pi - alpha, np.pi - alpha - beta]
    for idx, theta in enumerate(theta_list):
        S = np.sin(theta)
        C = np.cos(theta)
        
        den1 = np.sqrt( l + c*S*S )
        den2 = c*(1+C)+l+np.sqrt(l)*np.sqrt(l+c*S*S)
        den3 = (1-C)**2
        
        I[idx] = -0.5*np.sign(S)*( 2*np.sqrt(c)*np.arctan((np.sqrt(c)*C)/den1) ) + np.sqrt(l)\
                * np.log(((np.sqrt(l)*(1-2*c*C/(den2))))*(2*S*S/(den3)))
    return (-1/(4*np.pi))*abs(I[0] - I[1] - np.sqrt(c)*beta)


def genCoords(V, T, delta):    
    phi = np.zeros( (len(V), len(delta)) )
    psi = np.zeros( (len(T), len(delta)) )
    len_delta = len(delta)
    
    for eta_idx, eta in enumerate(delta):
        clear_output(wait=True)
        print(eta_idx+1,"/",len_delta," ", (eta_idx+1)/len_delta*100,"%")
        for j, face in enumerate(T):
            vj = face
            v = V[[vj[0], vj[1], vj[2]]]
            for l in range(3):
                v[l] -= eta 
            p = (v[0]@n[j])*n[j]
            s =  np.zeros(3)
            I =  np.zeros(3)
            II = np.zeros(3)
            q  = np.zeros((3,3))
            N = np.zeros((3,3))
            for l in range(3):
                s[l]  = np.sign( np.cross(v[l] - p, v[(l+1)%3] - p)@n[j])
                I[l]  = gcTriInt(p, v[l], v[(l+1)%3], np.zeros(3))
                II[l] = gcTriInt(np.zeros(3), v[(l+1)%3], v[l], np.zeros(3))
                q[l] = np.cross(v[(l+1)%3], v[l])
                N[l] = q[l]/np.linalg.norm(q[l])
                if(np.linalg.norm(q[l]) < EPS1): II[l] = 0 
            I_tot = -abs( (s[0]*I[0]) + (s[1]*I[1]) + (s[2]*I[2]) )
            psi[j][eta_idx] = -I_tot
            w = n[j]*I_tot + N[0]*II[0] + N[1]*II[1] + N[2]*II[2]
            if np.linalg.norm(w) > 0.00001:
                for l in range(3):
                    phi[vj[l], eta_idx] += (N[(l+1)%3]@w)/(N[(l+1)%3]@v[l])
            
    return phi, psi

phi, psi = genCoords(v_c, f_c, v_m)

1130 / 1130   100.0 %


**Some modification to the meshplot library are necessary since otherwise the camera is reset every time**

In [4]:
#Meshplot modifications
def gen_circle(width=256, height=256):
    xx, yy = np.mgrid[:width, :height]
    circle = (xx - width/2 + 0.5) ** 2 + (yy - height/2 + 0.5) ** 2
    array = np.ones((width, height, 4), dtype='float32')
    array[:, :, 0] = (circle <= width)
    array[:, :, 1] = (circle <= width)
    array[:, :, 2] = (circle <= width)
    array[:, :, 3] = circle <= width
    return array

def remove_object(viewer, obj_id):
    if obj_id not in viewer._Viewer__objects:
        print("Invalid object id. Valid ids are: ", list(viewer._Viewer__objects.keys()))
        return
    viewer._scene.remove(viewer._Viewer__objects[obj_id]["mesh"])
    del viewer._Viewer__objects[obj_id]
    
def add_object(viewer, obj, parent=None):
    if not parent: # Object is added to global scene and objects dict
        viewer._Viewer__objects[viewer._Viewer__cnt] = obj
        viewer._Viewer__cnt += 1
        viewer._scene.add(obj["mesh"])
    else: # Object is added to parent object and NOT to objects dict
        parent.add(obj["mesh"])

    return viewer._Viewer__cnt - 1

def add_line_geometry(viewer, lines, shading, obj=None):
    lines = lines.astype("float32", copy=False)
    mi = np.min(lines, axis=0)
    ma = np.max(lines, axis=0)

    geometry = p3s.LineSegmentsGeometry(positions=lines.reshape((-1, 2, 3)))
    material = p3s.LineMaterial(linewidth=shading["line_width"], color=shading["line_color"])
                #, vertexColors='VertexColors'),
    lines = p3s.LineSegments2(geometry=geometry, material=material) #type='LinePieces')
    line_obj = {"geometry": geometry, "mesh": lines, "material": material,
                "max": ma, "min": mi, "type": "Lines", "wireframe": None}

    if obj:
        return add_object(viewer, line_obj, obj), line_obj
    else:
        return add_object(viewer, line_obj)


def add_points(viewer, points, c=None, shading={}, obj=None, **kwargs):
    shading.update(kwargs)
    if len(points.shape) == 1:
        if len(points) == 2:
            points = np.array([[points[0], points[1], 0]])
    else:
        if points.shape[1] == 2:
            points = np.append(
                points, np.zeros([points.shape[0], 1]), 1)
    sh = viewer._Viewer__get_shading(shading)
    points = points.astype("float32", copy=False)
    mi = np.min(points, axis=0)
    ma = np.max(points, axis=0)

    g_attributes = {"position": p3s.BufferAttribute(points, normalized=False)}
    m_attributes = {"size": sh["point_size"]}

    if sh["point_shape"] == "circle": # Plot circles
        tex = p3s.DataTexture(data=gen_circle(16, 16), format="RGBAFormat", type="FloatType")
        m_attributes["map"] = tex
        m_attributes["alphaTest"] = 0.5
        m_attributes["transparency"] = True
    else: # Plot squares
        pass

    colors, v_colors = viewer._Viewer__get_point_colors(points, c, sh)
    if v_colors: # Colors per point
        m_attributes["vertexColors"] = 'VertexColors'
        g_attributes["color"] = p3s.BufferAttribute(colors, normalized=False)

    else: # Colors for all points
        m_attributes["color"] = colors

    material = p3s.PointsMaterial(**m_attributes)
    geometry = p3s.BufferGeometry(attributes=g_attributes)
    points = p3s.Points(geometry=geometry, material=material)
    point_obj = {"geometry": geometry, "mesh": points, "material": material,
        "max": ma, "min": mi, "type": "Points", "wireframe": None}

    if obj:
        return add_object(viewer, point_obj, obj), point_obj
    else:
        return add_object(viewer, point_obj)
    
def add_lines(viewer, beginning, ending, shading={}, obj=None, **kwargs):
    shading.update(kwargs)
    if len(beginning.shape) == 1:
        if len(beginning) == 2:
            beginning = np.array([[beginning[0], beginning[1], 0]])
    else:
        if beginning.shape[1] == 2:
            beginning = np.append(
                beginning, np.zeros([beginning.shape[0], 1]), 1)
    if len(ending.shape) == 1:
        if len(ending) == 2:
            ending = np.array([[ending[0], ending[1], 0]])
    else:
        if ending.shape[1] == 2:
            ending = np.append(
                ending, np.zeros([ending.shape[0], 1]), 1)

    sh = viewer._Viewer__get_shading(shading)
    lines = np.hstack([beginning, ending])
    lines = lines.reshape((-1, 3))
    return add_line_geometry(viewer, lines, sh, obj)

# User interface

In [20]:
selected_is_empty = True 
edge_obj = 2
green_points_obj = 3
red_points_obj = -1 #Not created yet
x_range, y_range, z_range  = np.max(v_c, axis=0) - np.min(v_c, axis=0) 

grid = iw.GridspecLayout(4, 2)

def drawPoints():
    #global selected_is_empty
    global selected_is_empty, green_points_obj, edge_obj, red_points_obj
    
    remove_object(ui, green_points_obj)
    if not selected_is_empty: remove_object(ui, red_points_obj)
    not_selected = np.where(current_selected == 0)
    selected_points = np.where(current_selected == 1)
    #Add not selected points
    add_points(ui, v_cc[not_selected], shading={"point_color": "green", "point_size": 0.5})
    green_points_obj = np.max([edge_obj, green_points_obj, red_points_obj]) + 1
    #Add selected points
    if np.size(selected_points) > 0:
        add_points(ui, v_cc[selected_points], shading={"point_color": "blue", "point_size": 0.5})
        selected_is_empty = False
        red_points_obj = np.max([edge_obj, green_points_obj, red_points_obj]) + 1
    else: selected_is_empty = True

    
def displacePoints():
    #global selected_is_empty
    global selected_is_empty, green_points_obj, edge_obj, red_points_obj
    
    #remove current cage
    remove_object(ui, edge_obj)
    if not selected_is_empty: remove_object(ui, red_points_obj)
    not_selected = np.where(current_selected == 0)
    selected_points = np.where(current_selected == 1)
    #Add selected points
    if np.size(selected_points) > 0:
        add_points(ui, v_cc[selected_points], shading={"point_color": "blue", "point_size": 0.5})
        selected_is_empty = False
        red_points_obj = np.max([edge_obj, green_points_obj, red_points_obj]) + 1
    else: selected_is_empty = True
    #Add edges
    lines_start = np.vstack( [v_cc[f_c[:,0]], v_cc[f_c[:,1]], v_cc[f_c[:,2]]] )
    lines_end =  np.vstack( [v_cc[f_c[:,1]], v_cc[f_c[:,2]], v_cc[f_c[:,0]]] )
    add_lines(ui, lines_start, lines_end, shading={"line_color": "red"})
    edge_obj = np.max([edge_obj, green_points_obj, red_points_obj]) + 1 
            
#interface
v_ref = np.copy(v_c)
v_cc = np.copy(v_c) #current cage

current_selected = np.zeros(len(v_c), dtype=int)
displ = np.zeros([len(v_c),3])

select_button   = iw.Button(description="Select")
deselect_button = iw.Button(description="Deselect")
deselect_all_button = iw.Button(description="Deselect all")
clear_button = iw.Button(description="Clear displace sliders")

# Set Callback
def select_clicked(b):
    new_points = np.where(np.linalg.norm(v_cc - sf.coord[1:],axis=1) < sf.coord[0])[0]
    current_selected[new_points] = 1
    drawPoints()
    
def deselect_clicked(b):
    new_points = np.where(np.linalg.norm(v_cc - sf.coord[1:],axis=1) < sf.coord[0])[0]
    current_selected[new_points] = 0
    drawPoints()
    
def deselect_all_clicked(b):
    current_selected[:] = 0
    drawPoints()
    
def clear_clicked(b):
    global current_selected
    tmp = np.copy(current_selected)
    current_selected[:] = 0
    grid[1,1].value = 0
    grid[2,1].value = 0
    grid[3,1].value = 0
    current_selected = tmp
    drawPoints()
    v_ref[:] = v_cc[:] 
    displ[:] = 0
    
select_button.on_click(select_clicked)
deselect_button.on_click(deselect_clicked)
deselect_all_button.on_click(deselect_all_clicked)
clear_button.on_click(clear_clicked)


# Meshplot
ui = mp.plot(v_m, f_m)
ui.add_mesh(v_s*0.1, f_s, c=np.array([1,0,0]))
lines_start = np.vstack( [v_cc[f_c[:,0]], v_cc[f_c[:,1]], v_cc[f_c[:,2]]] )
lines_end =  np.vstack( [v_cc[f_c[:,1]], v_cc[f_c[:,2]], v_cc[f_c[:,0]]] )
ui.add_lines(lines_start, lines_end, shading={"line_color": "red"})
ui.add_points(v_cc, shading={"point_color": "green", "point_size": 0.5})

# Display Buttons
display(iw.HBox([select_button, deselect_button, deselect_all_button, clear_button]))

#Selection ball
def sf(radius,x,y,z):
    ui.update_object(oid = 1, vertices = v_s*radius + np.array([x,y,z]))
    #ui.remove_object(10)
    sf.coord = [radius,x,y,z]
    

grid[0,0] = iw.FloatSlider(min=0.1, max=1, value=0.1, description='Radius')
grid[1,0] = iw.FloatSlider(min=-2.5, max=2.5, value=0, description='x')
grid[2,0] = iw.FloatSlider(min=-2.5, max=2.5, value=0, description='y')
grid[3,0] = iw.FloatSlider(min=-2.5, max=2.5, value=0, description='z')

iw.interactive_output(sf, {
            'radius': grid[0,0],
            'x': grid[1,0],
            'y': grid[2,0],
            'z': grid[3,0]
})

#Displacements

grid[1,1] = iw.FloatSlider(min=-x_range/3, max=x_range/3, value=0, description='displace_x')
grid[2,1] = iw.FloatSlider(min=-y_range/3, max=y_range/3, description='displace_y')
grid[3,1] = iw.FloatSlider(min=-z_range/3, max=z_range/3, description='displace_z')

def displace(displace_x,displace_y,displace_z):
    global v_cc
    current_displ = np.array([displace_x,displace_y,displace_z])
    displ[current_selected == 1] = current_displ
    v_cc = v_ref + displ
    
    if not selected_is_empty: 
        displacePoints()
        #update v 
        deformed = - (phi.T@v_cc + psi.T@n)
        ui.update_object(oid = 0, vertices = deformed)
    return




iw.interactive_output(displace, {
            'displace_x': grid[1,1],
            'displace_y': grid[2,1],
            'displace_z': grid[3,1]
})

grid

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.317513…

GridspecLayout(children=(FloatSlider(value=0.1, description='Radius', layout=Layout(grid_area='widget001'), ma…